In [30]:
import pandas as pd
import numpy as np
import datetime 

version = '2.0.3'

fn_wind_unit = f'data/bnetza_mastr_rli_v{version}_unit-wind'
fn_wind = f'data/bnetza_mastr_rli_v{version}_wind'
fn_wind_eeg = f'data/bnetza_mastr_rli_v{version}_wind-eeg'

df_wind = pd.read_csv(f'{fn_wind}.csv', 
                      encoding='utf8', sep=';', dtype={"Postleitzahl":str})

df_wind_unit = pd.read_csv(f'{fn_wind_unit}.csv', 
                      encoding='utf8', sep=';', dtype={"Postleitzahl":str})
df_wind_eeg = pd.read_csv(f'{fn_wind_eeg}.csv', 
                      encoding='utf8', sep=';', dtype={"Postleitzahl":str})

In [31]:
""" from pandas github https://github.com/pandas-dev/pandas/issues/11250 """
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cs = frame[v].columns
        vs = frame[v]
        lcs = len(cs)

        for i in range(lcs):
            iv = vs.iloc[:,i].tolist()
            for j in range(i+1, lcs):
                jv = vs.iloc[:,j].tolist()
                if iv == jv:
                    dups.append(cs[i])
                    break

    return dups

In [32]:
import datetime

df_wind.EinheitMastrNummer.astype(str)
df_wind_unit.EinheitMastrNummer.astype(str)

df_wind = df_wind.drop(['EegMastrNummer','timestamp','lid'], axis=1)
df_all = df_wind.merge(df_wind_unit, on='EinheitMastrNummer')
df_all.merge(df_wind_eeg, on='EegMastrNummer')
df_all['timestamp'] = datetime.datetime.now()
df_all.to_csv("all_windunits.csv", index=False)

""" NOW, check for other duplicates considering SME / SEE and PLZ """

""" then consider Leistung and Meldedatum """

' then consider Leistung and Meldedatum '

In [33]:
def duplicate_filtering(meldedatum, mastrnr, aktualisierung):
    if pd.Series.to_string(mastrnr).find('SEE'):
        init_date = datetime.datetime(2019, 1, 31)
        if not meldedatum.isnull().any():
            for x in meldedatum:
                dt = datetime.datetime.strptime(x,'%Y-%m-%d')
                if dt < datetime.datetime.strptime('2019-01-31','%Y-%m-%d'):
                    print(dt)

split = df_all.groupby(['Postleitzahl']).apply(lambda x : 
                                                                    duplicate_filtering(x['Meldedatum'], 
                                                                                   x['EinheitMastrNummer'],
                                                                                   x['DatumLetzteAktualisierung']))

In [34]:
""" Test for Meldedatum """
df_allWithMeldedatum = df_all.loc[~df_all['Meldedatum'].isnull()]
df_allWithMeldedatum.to_csv("all_wind_with_Meldedatum.csv", index=False)
df_MeldedatumSME = df1.loc[pd.to_datetime(df1['Meldedatum']) < datetime.datetime.strptime('2019-01-31','%Y-%m-%d')]
df_MeldedatumSME.head()

""" --> 1) not enough entries with non NaN Meldedatum 2) no Meldedatum < 31.01.19 """

' --> 1) not enough entries with non NaN Meldedatum 2) no Meldedatum < 31.01.19 '

In [35]:
""" Test for DatumLetzteAktualisierung """

df_allWithUpdate = df_all.loc[~df_all['DatumLetzteAktualisierung'].isnull()]



""" SME < 31.1.19 """

df_UpdateAndDateSME = df_allWithUpdate.loc[pd.to_datetime(df_allWithUpdate['DatumLetzteAktualisierung']) 
                           <= datetime.datetime.strptime('2019-01-31','%Y-%m-%d')]
""" check wether EinheitMastrNummer is SME """
df_SME = df_UpdateAndDateSME[df_UpdateAndDateSME['EinheitMastrNummer'].astype(str).str.contains('SME')]



""" SEE > 31.1.19 """

df_UpdateAndDateSEE = df_allWithUpdate.loc[pd.to_datetime(df_allWithUpdate['DatumLetzteAktualisierung']) >
                           datetime.datetime.strptime('2019-01-31','%Y-%m-%d')]
""" check wether EinheitMastrNummer is SME """
df_SEE = df_UpdateAndDateSEE[df_UpdateAndDateSEE['EinheitMastrNummer'].astype(str).str.contains('SEE')]

In [36]:
""" test for duplicates based on EinheitMastrNummer, date divergence, same plz and leistung """    

def check_dups(df, plz):
    df_SME = df[df['EinheitMastrNummer'].astype(str).str.contains('SME')]
    df_SEE = df[df['EinheitMastrNummer'].astype(str).str.contains('SEE')]
    if not df_SME.empty and not df_SEE.empty:
        """ NOW search for duplicate leistungen -- for elem in df : search for same Leistung in other df"""
        df = pd.DataFrame()
        df_new = pd.DataFrame()
        """ compare each SME Leistung with each Leistung in SEE """
        for name,val in df_SME.iterrows():
            mastrNR = df_SEE['EinheitMastrNummer'].where(df_SEE['Bruttoleistung_y']==val['Bruttoleistung_y'])
            if not mastrNR.isnull().any():
                df_new = pd.DataFrame({'SME_nr':val['EinheitMastrNummer'], 'SEE_nr':mastrNR, 'bruttoleistung':val['Bruttoleistung_y'], 'plz': plz})
            if not df_new.empty:
                df = df.append(df_new)
        return df

In [37]:
""" NOW - we have two frames: one with SME <= 31.1.19 and one with SEE > 31.1.19 """

pd_all = pd.concat([df_SEE, df_SME]).groupby(['Postleitzahl'], as_index=False)
all_dups = pd.DataFrame()

for plz, df in pd_all:
    all_dups = all_dups.append(check_dups(df, plz))

all_dups.to_csv("all_dups.csv")

""" total length of concat array: 25059 // plz with SEE and SME: 16323 """
all_dups.to_csv("possible_duplicates.csv")